In [1]:
import numpy as np
import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
from numba import njit , prange
from matplotlib.colors import TwoSlopeNorm
import progressbar
import hints
from itertools import permutations
import matplotlib.gridspec as gridspec
import matplotlib
from matplotlib.ticker import AutoMinorLocator
from scipy.io import loadmat
import glob
import imageio.v2 as imageio
import scipy

In [2]:
from jitcsde import jitcsde, y, t


In [3]:
def KMC_ABC(x,dt,N,M,mode,method):
    
    # if method=='abolfazl':
    #     x, y = caly(x, N)
    #     Dcoeffs, ind, col = calD1coef(x, y, dt, N, M, mode)
    # elif method=='hints':
    #     calulator = hints.kmcc(ts_array=np.swapaxes(x,0,1), dt=dt, interaction_order=M, estimation_mode='drift')
    #     Dcoeffs = calulator.get_coefficients()  
    # elif method=='hints2' or method=='amin':   
    #     Dcoeffs = kmc(np.swapaxes(x,0,1), dt=dt, order=[1,2,3], mode='drift', **{'window_order':5, 'solver':'both', 'threshold':1e-15})   # 0
    # else:
    #     raise ValueError("Method options are: 'abolfazl', 'hints', 'hints2' ")

    calulator = hints.kmcc(ts_array=np.swapaxes(x,0,1), dt=dt, interaction_order=M, estimation_mode='drift')
    Dcoeffs = calulator.get_coefficients()
    Dcoeffs = np.swapaxes(Dcoeffs.to_numpy(), 0, 1)

    Constant = np.zeros(N)
    A = np.zeros((N, N))        
    B = np.zeros((N, N, N))    
    C = np.zeros((N, N, N, N))  

    
    num_first_order = N
    num_second_order = int(N * (N + 1) / 2)
    num_third_order = int(N * (N + 1) * (N + 2) / 6)

    
    if 0 in M:  
        Constant[:] = Dcoeffs[:, 0]
        A[:, :] = Dcoeffs[:, 1:N+1]  
    else:
        A[:, :] = Dcoeffs[:, :N]     
    
    
    if 2 in M:  
        if 0 in M:
            counter = num_first_order + 1
        else:    
            counter = num_first_order
        for i in range(N):
            for j in range(i, N):
                for idx in set(permutations((i, j))): B[:, idx[0], idx[1]] = Dcoeffs[:, counter]
                counter += 1

    if 3 in M:  
        if 0 in M:
            counter = num_first_order + num_second_order + 1
        else:
            counter = num_first_order + num_second_order
        for i in range(N):
            for j in range(i, N):
                for k in range(j, N):
                    for idx in set(permutations((i, j, k))): C[:, idx[0], idx[1], idx[2]] = Dcoeffs[:, counter]
                    counter += 1

    return Constant, A, B, C, Dcoeffs

def KMC_ABC_overtime(data,win,ol,N,M,mode,dt,tstart,zmean,uvar,method):
    dx=win*(100-ol)/100
    n_win=int((data.shape[1]-win)/dx+1)
    alleval=[]
    allevec=[]
    allcons=np.zeros((n_win,N))
    alla=np.zeros((n_win,N,N))
    allb=np.zeros((n_win,N,N,N))
    allc=np.zeros((n_win,N,N,N,N))
    alldcoefllist=[]
    t=int(win/2+tstart)
    tlist=[]
    ii=0
    tarray=np.arange(int(win / 2 + tstart), int(data.shape[1] - win / 2), dx)

    # bar = progressbar.ProgressBar(maxval=len(tarray),  redirect_stdout=True,  redirect_stderr=True, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    # bar.start()


    for t in tarray:
        tlist.append(t)
        start=int(t-win/2)
        end=int(t+win/2)
        x=np.zeros((N,win))
        x[:,:]=data[:,start:end]

        for i in range(N):
            if zmean==True:
                x[i,:]=x[i,:]-np.mean(x[i,:])
            if uvar==True:
                x[i,:]=x[i,:]/np.std(x[i,:])

        cons, A, B, C, Dcoeffs=KMC_ABC(x,dt,N,M,mode,method)
        allcons[ii,:]=cons[:]
        alla[ii,:,:]=A[:,:]
        allb[ii,:,:,:]=B[:,:,:]
        allc[ii,:,:,:,:]=C[:,:,:,:]
        alldcoefllist.append(Dcoeffs)
        
        w, v = np.linalg.eig(A)
        alleval.append(w)
        allevec.append(v)
        ii+=1
    #     bar.update(ii)
    # bar.finish()

    alldcoefllist=np.array(alldcoefllist)
    alleval=np.array(alleval)
    allevec=np.array(allevec)


    return tlist, allcons, alla, allb, allc, alldcoefllist, alleval, allevec

In [6]:
data_L10_8 = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/Lorenz96_10D Data & Coef/Lorenz96_10D_noise0.7_dt0.005_F8.npy")
data_L10_8.shape


(10, 5000001)

In [11]:
dt=0.005
N=10
M=[0, 1, 2]
mode='list'
win=50000
ol=0
tstart=0
zmean=False
uvar=False
method='hints'
tlist, allcons_L10_8, alla_L10_8, allb_L10_8, allc, alldcoefllist, alleval, allevec=KMC_ABC_overtime(data_L10_8,win,ol,N,M,mode,dt,tstart,zmean,uvar,method)
np.save(f"Lorenz96_10D_L10_8_A_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", alla_L10_8)
np.save(f"Lorenz96_10D_L10_8_B_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", allb_L10_8)
np.save(f"Lorenz96_10D_L10_8_Const_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", allcons_L10_8)

In [12]:
A_L10_8_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/Lorenz96_10D Data & Coef/Lorenz96_10D_L10_8_A_win=50000_overlap=0_mean=False_unitvar=False.npy")
B_L10_8_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/Lorenz96_10D Data & Coef/Lorenz96_10D_L10_8_B_win=50000_overlap=0_mean=False_unitvar=False.npy")

Const_L10_8_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/Lorenz96_10D Data & Coef/Lorenz96_10D_L10_8_Const_win=50000_overlap=0_mean=False_unitvar=False.npy")


In [15]:
win_num = len(Const_L10_8_all)
print(win_num)
print(Const_L10_8_all.shape)


100
(100, 10)


In [30]:
from scipy import stats

def MeanWithCutoff_Apply(win_los, M):
    win_num = len(M)
    M_new = np.copy(M[win_los:win_num])
    M_mean = np.mean(M_new, axis=0)
    M_std = np.std(M_new, axis=0, ddof=1)
    M_cut = np.copy(M_mean)
    M_cut[(abs(M_mean) < stats.t.ppf(0.95, win_num-win_los)*abs(M_std/np.sqrt(win_num-win_los)))] = 0
    return M_cut

def MedianWithCutoff_Apply(win_los, M):
    win_num = len(M)
    M_new = np.copy(M[win_los:win_num])
    M_median = np.median(M_new, axis=0)
    M_std = np.std(M_new, axis=0, ddof=1)
    M_cut = np.copy(M_median)
    M_cut[(abs(M_median) < stats.t.ppf(0.95, win_num-win_los)*abs(M_std/np.sqrt(win_num-win_los)))] = 0
    return M_cut


In [31]:
from jitcsde import jitcsde, y, t
def jit_equation_0_3(Const, A, B, C):
    n = A.shape[0]
    eq = []
    for i in range(n):
        eq.append(Const[i] + sum(A[i,j] * y(j) for j in range(n)) + sum(B[i,j,k] * y(j) * y(k) for j in range(n) for k in range(j,n)) + sum(C[i,j,k,l] * y(j) * y(k) * y(l) for j in range(n) for k in range(j,n) for l in range(j,k,n)))
    return eq


from jitcsde import jitcsde, y, t
def jit_equation_0_2(Const, A, B):
    n = A.shape[0]
    eq = []
    for i in range(n):
        eq.append(Const[i] + sum(A[i,j] * y(j) for j in range(n)) + sum(B[i,j,k] * y(j) * y(k) for j in range(n) for k in range(j,n)))
    return eq




In [23]:
def save_equations_to_tt(equations, filename):
    with open(filename, 'w') as f:
        for i, eq in enumerate(equations):
            # f.write(f"Equation {i+1}:\n")
            f.write(str(eq))
            f.write("\n\n")

In [32]:
jit_eq = jit_equation_0_2(MeanWithCutoff_Apply(2, Const_L10_8_all )  , MeanWithCutoff_Apply(2, A_L10_8_all), MeanWithCutoff_Apply(2, B_L10_8_all))
jit_eq
# str(jit_eq) 


[8.19461536052336 + 0.00326025481126608*y(0)*y(1) + 0.005136139017728*y(0)*y(2) + 0.00380532964935848*y(0)*y(3) + 0.00162628022496527*y(0)*y(4) - 0.00108546219558691*y(0)*y(5) + 0.0014388164972734*y(0)*y(6) - 0.00334543445163754*y(0)*y(7) - 0.00254555371611417*y(0)*y(8) - 0.0071901799318943*y(0)*y(9) - 0.00130308583747179*y(1)*y(5) - 0.00259276337904868*y(1)*y(8) + 0.998632089387608*y(1)*y(9) - 0.00141357226202825*y(2)*y(8) + 0.00405706487235998*y(2)*y(9) - 0.00114604646375569*y(3)*y(5) + 0.00224216558947586*y(3)*y(8) + 0.00189387505333487*y(3)*y(9) + 0.00109239619775094*y(4)*y(5) + 0.00438874429495593*y(6)*y(8) + 0.00381781169674266*y(7)*y(6) + 0.00167887102769595*y(9)*y(6) - 1.00008165433188*y(9)*y(8) + 0.00348481946689238*y(0)**2 + 0.00290925649468928*y(1)**2 + 0.00092503752292039*y(2)**2 + 0.00242610740155571*y(7)**2 + 0.00653756961532309*y(8)**2 - 0.00675542198141316*y(9)**2 - 1.09017419755054*y(0) - 0.01415338899855*y(1) - 0.0264734475487377*y(2) - 0.0159199895413493*y(3) - 0.019

In [33]:
with open("jit_equations_L10_8.txt", 'w') as f:
    f.write(str(jit_eq))